# Introduction
Let's use dimensionality reduction in a real example.
We will examine the MNIST database of digits, which has a large number of features 28x28. We reduce the dimensionality of our dataset to then use our preferred algorithm to predict the data.

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import seaborn as sns
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

# Import the 3 dimensionality reduction methods
#class sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

# MNIST Dataset
We choose the popular MNIST (Mixed National Institute of Standards and Technology) computer vision digit dataset. This contains a series for images of handwriting letters, each of them of 28x28 pixels, see below:

In [2]:
train = pd.read_csv('../datasets/mnist_train.csv').head(3000) 
#reduce the size to 3000 to make things fast

columns=[]
columns.append("label")
for ii in range(784):
    columns.append("pixel"+str(ii+1))
train.columns=columns
               
train.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
print(train.shape)


(3000, 785)


The MNIST set consists of 59999 rows and 785 columns. There are 28 x 28 pixel images of digits ( contributing to 784 columns) as well as one extra label column which is essentially a class label to state whether the row-wise contribution to each digit gives a 1 or a 9. 
Plot an example:

In [4]:
# Copy the features and target columns to different arrays: 
y_train= train['label']
# Drop the label feature
X_train = train.drop("label",axis=1)


Now having calculated both our Individual Explained Variance and Cumulative Explained Variance values, let's use the Plotly visualisation package to produce an interactive chart to showcase this.

In [5]:
from IPython.display import display, Math, Latex

lda = LDA(n_components=20)
# Taking in as second argument the Target as labels
lda = lda.fit(X_train.values, y_train.values )
X_train_lda = lda.transform(X_train.values)

print(X_train_lda.shape)

(3000, 9)


/Users/rangel/Virtualenvs/pip3-venv/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:387: UserWarning:

Variables are collinear.



In [6]:
# Using the Plotly library again
traceLDA = go.Scatter(
    x = X_train_lda[:,0],
    y = X_train_lda[:,1],
    name = y_train,
#     hoveron = Target,
    mode = 'markers',
    text = y_train,
    showlegend = True,
    marker = dict(
        size = 8,
        color = y_train,
        colorscale ='Jet',
        showscale = False,
        line = dict(
            width = 2,
            color = 'rgb(255, 255, 255)'
        ),
        opacity = 0.8
    )
)
data = [traceLDA]

layout = go.Layout(
    title= 'Linear Discriminant Analysis (LDA)',
    hovermode= 'closest',
    xaxis= dict(
         title= 'First Linear Discriminant',
        ticklen= 5,
        zeroline= False,
        gridwidth= 2,
    ),
    yaxis=dict(
        title= 'Second Linear Discriminant',
        ticklen= 5,
        gridwidth= 2,
    ),
    showlegend= False
)

fig = dict(data=data, layout=layout)
py.iplot(fig, filename='styled-scatter')

In [7]:
test = pd.read_csv('../datasets/mnist_test.csv')
test.columns=columns
               
test.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
# save the labels to a Pandas series target
y_test = test['label']
# Drop the label feature
X_test = test.drop("label",axis=1)

Now we use the LDA object to predict in the test set.


Remember the precision is the ratio tp / (tp + fp) where tp is the number of true positives and fp the number of false positives. The precision is intuitively the ability of the classifier not to label as positive a sample that is negative.

In [9]:
#Total of true positives:
from sklearn.metrics import precision_score

y_pred = lda.predict(X_test.values) 

#Precision score for test dataset:
print("Precision score for test dataset: \n")
precision_score(y_test, y_pred, average='micro')

Precision score for test dataset: 



0.81108110811081113

In [10]:
X_test_lda = lda.transform(X_test) 


Now lets compare this performance to that of svm methods: linear and polynomial

In [11]:
from sklearn import svm

from sklearn.metrics import classification_report

estimator = svm.LinearSVC(C=1.0)

estimator.fit(X_train_lda, y_train) 

y_pred = estimator.predict(X_test_lda)

ntest=y_pred.size


#Precision score for test dataset:
print("Precision score for test dataset: \n")
precision_score(y_test, y_pred, average='micro')


Precision score for test dataset: 



0.8101810181018102

In [12]:
# Import
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(degree=2)
X_train_poly = poly.fit_transform(X_train_lda)
X_test_poly=poly.fit_transform(X_test_lda)

estimator.fit(X_train_poly, y_train)
y_pred = estimator.predict(X_test_poly)

#Precision score for test dataset:
print("Precision score for test dataset: \n")
precision_score(y_test, y_pred, average='micro')



Precision score for test dataset: 



0.77237723772377243

In this case the polynomial regression performs worse than the linear regression